In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math

In [2]:
#define current city
city = 'newberg'

In [8]:
#read in labels
labels = pd.read_csv('../../../data/{}/processed-labels/labels_raw_{}.csv'.format(city, city, city))
labels

,label_id,gsv_panorama_id_x,label_type,severity,correct,has_validations,expired,high_quality_user,geometry,gsv_panorama_id_y,pano_width,pano_height,pano_x,pano_y,zoom,heading,pitch,camera_heading,camera_pitch,user_id,lat,lng
0,145,0R2JTLibSCNsNb7aGNUzWg,NoSidewalk,NaN,NaN,False,True,True,POINT (-122.9529037475586 45.317649841308594),0R2JTLibSCNsNb7aGNUzWg,13312.0,6656.0,9123,3863,1,319.375000,-19.062500,269.813477,0.678093,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317650,-122.952904
1,18134,Z6Ja9-4wJxpQZPEnn8KSXA,NoSidewalk,4.0,NaN,False,False,True,POINT (-122.96400451660156 45.313819885253906),Z6Ja9-4wJxpQZPEnn8KSXA,13312.0,6656.0,6644,3744,3,326.437500,-10.906250,323.408020,1.379303,bb78d1ab-85d6-4620-a1de-396a5785080a,45.313820,-122.964005
2,151,oqg9ULHZul23xfU6DwTRNg,NoSidewalk,NaN,NaN,False,False,True,POINT (-122.95359802246094 45.3176155090332),oqg9ULHZul23xfU6DwTRNg,13312.0,6656.0,7788,3776,2,285.258942,-11.508928,268.211182,0.926697,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317616,-122.953598
3,153,WWRq8hx_qzBmTiSkSBjEqQ,NoSidewalk,NaN,NaN,False,True,True,POINT (-122.95402526855469 45.317420959472656),WWRq8hx_qzBmTiSkSBjEqQ,13312.0,6656.0,4663,3709,2,223.651779,-8.741072,268.896179,-0.547432,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317421,-122.954025
4,154,WWRq8hx_qzBmTiSkSBjEqQ,NoSidewalk,NaN,NaN,False,True,True,POINT (-122.95418548583984 45.31760025024414),WWRq8hx_qzBmTiSkSBjEqQ,13312.0,6656.0,7268,3600,2,279.008942,-9.455358,268.896179,-0.547432,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317600,-122.954185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16842,8222,5KKnsHQgODUBbIRZYiVbqg,NoSidewalk,NaN,NaN,False,False,True,POINT (-122.92579650878906 45.30332946777344),5KKnsHQgODUBbIRZYiVbqg,13312.0,6656.0,256,3757,2,225.142853,-19.375000,22.380564,-3.511421,fc36440f-38b7-4060-892a-65fedd77f651,45.303329,-122.925797
16843,8272,mdbgEoi-XTfZGiKqcWMlqw,NoSidewalk,NaN,NaN,False,True,True,POINT (-122.92559051513672 45.29985809326172),mdbgEoi-XTfZGiKqcWMlqw,3328.0,1664.0,2697,986,1,114.982140,-15.312500,359.096985,-9.832024,fc36440f-38b7-4060-892a-65fedd77f651,45.299858,-122.925591
16844,8281,BtyhcErvfI4z31FisBjeQg,NoSidewalk,NaN,NaN,False,False,True,POINT (-122.92660522460938 45.305442810058594),BtyhcErvfI4z31FisBjeQg,13312.0,6656.0,7561,3633,1,43.812500,-1.875000,46.910660,-2.319252,fc36440f-38b7-4060-892a-65fedd77f651,45.305443,-122.926605
16845,16595,--gG93FA53Lkv0H31-9Ruw,CurbRamp,NaN,True,True,False,True,POINT (-122.95636749267578 45.29937744140625),--gG93FA53Lkv0H31-9Ruw,13312.0,6656.0,1748,3983,1,142.982147,-11.625000,270.257080,-1.495285,fc36440f-38b7-4060-892a-65fedd77f651,45.299377,-122.956367


In [13]:
#select only the columns we need
labels = labels[['label_id','label_type','severity','gsv_panorama_id','zoom','heading','pitch','camera_heading','camera_pitch','user_id','lat','lng', 'geometry']]

In [14]:
labels

,label_id,label_type,severity,gsv_panorama_id,zoom,heading,pitch,camera_heading,camera_pitch,user_id,lat,lng,geometry
0,145,NoSidewalk,NaN,0R2JTLibSCNsNb7aGNUzWg,1,319.375000,-19.062500,269.813477,0.678093,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317650,-122.952904,POINT (-122.9529037475586 45.317649841308594)
1,18134,NoSidewalk,4.0,Z6Ja9-4wJxpQZPEnn8KSXA,3,326.437500,-10.906250,323.408020,1.379303,bb78d1ab-85d6-4620-a1de-396a5785080a,45.313820,-122.964005,POINT (-122.96400451660156 45.313819885253906)
2,151,NoSidewalk,NaN,oqg9ULHZul23xfU6DwTRNg,2,285.258942,-11.508928,268.211182,0.926697,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317616,-122.953598,POINT (-122.95359802246094 45.3176155090332)
3,153,NoSidewalk,NaN,WWRq8hx_qzBmTiSkSBjEqQ,2,223.651779,-8.741072,268.896179,-0.547432,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317421,-122.954025,POINT (-122.95402526855469 45.317420959472656)
4,154,NoSidewalk,NaN,WWRq8hx_qzBmTiSkSBjEqQ,2,279.008942,-9.455358,268.896179,-0.547432,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317600,-122.954185,POINT (-122.95418548583984 45.31760025024414)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16842,8222,NoSidewalk,NaN,5KKnsHQgODUBbIRZYiVbqg,2,225.142853,-19.375000,22.380564,-3.511421,fc36440f-38b7-4060-892a-65fedd77f651,45.303329,-122.925797,POINT (-122.92579650878906 45.30332946777344)
16843,8272,NoSidewalk,NaN,mdbgEoi-XTfZGiKqcWMlqw,1,114.982140,-15.312500,359.096985,-9.832024,fc36440f-38b7-4060-892a-65fedd77f651,45.299858,-122.925591,POINT (-122.92559051513672 45.29985809326172)
16844,8281,NoSidewalk,NaN,BtyhcErvfI4z31FisBjeQg,1,43.812500,-1.875000,46.910660,-2.319252,fc36440f-38b7-4060-892a-65fedd77f651,45.305443,-122.926605,POINT (-122.92660522460938 45.305442810058594)
16845,16595,CurbRamp,NaN,--gG93FA53Lkv0H31-9Ruw,1,142.982147,-11.625000,270.257080,-1.495285,fc36440f-38b7-4060-892a-65fedd77f651,45.299377,-122.956367,POINT (-122.95636749267578 45.29937744140625)


In [15]:
#read in all files in the feature folder
clustered = pd.read_csv('../../../data/' + city + '/features/clustered.csv')
distance_to_road = pd.read_csv('../../../data/' + city + '/features/distance_to_road.csv')
distance_to_intersection = pd.read_csv('../../../data/' + city + '/features/distance_to_intersection.csv')
optional_input = pd.read_csv('../../../data/' + city + '/features/optional_input.csv')

In [16]:
#merge all into one on label_id
df = labels.merge(clustered, on='label_id').merge(distance_to_road, on='label_id').merge(distance_to_intersection, on='label_id').merge(optional_input, on='label_id')

In [17]:
#keep only the label types Curb Ramp, Missing Curb Ramp, Obstacle in Path, Surface Problem, and No Sidewalk
df = df[df['label_type'].isin(['CurbRamp', 'NoCurbRamp', 'Obstacle', 'SurfaceProblem', 'NoSidewalk'])]

In [18]:
df

,label_id,label_type,severity,gsv_panorama_id,zoom,heading,pitch,camera_heading,camera_pitch,user_id,lat,lng,geometry,cluster_id,clustered,cluster_label_count,way_type,distance_to_road,distance_to_intersection,tag,tag_count,description
0,145,NoSidewalk,NaN,0R2JTLibSCNsNb7aGNUzWg,1,319.375000,-19.062500,269.813477,0.678093,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317650,-122.952904,POINT (-122.9529037475586 45.317649841308594),13284,1,2,tertiary,21.228372,0.000000,0.0,0.0,0.0
1,18134,NoSidewalk,4.0,Z6Ja9-4wJxpQZPEnn8KSXA,3,326.437500,-10.906250,323.408020,1.379303,bb78d1ab-85d6-4620-a1de-396a5785080a,45.313820,-122.964005,POINT (-122.96400451660156 45.313819885253906),10678,1,2,tertiary,36.874003,0.000000,1.0,3.0,0.0
2,151,NoSidewalk,NaN,oqg9ULHZul23xfU6DwTRNg,2,285.258942,-11.508928,268.211182,0.926697,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317616,-122.953598,POINT (-122.95359802246094 45.3176155090332),6734,0,1,secondary,34.955005,0.000000,0.0,0.0,0.0
3,153,NoSidewalk,NaN,WWRq8hx_qzBmTiSkSBjEqQ,2,223.651779,-8.741072,268.896179,-0.547432,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317421,-122.954025,POINT (-122.95402526855469 45.317420959472656),13312,0,1,secondary,34.211111,0.000000,0.0,0.0,0.0
4,154,NoSidewalk,NaN,WWRq8hx_qzBmTiSkSBjEqQ,2,279.008942,-9.455358,268.896179,-0.547432,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317600,-122.954185,POINT (-122.95418548583984 45.31760025024414),13317,0,1,secondary,31.433333,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16842,8222,NoSidewalk,NaN,5KKnsHQgODUBbIRZYiVbqg,2,225.142853,-19.375000,22.380564,-3.511421,fc36440f-38b7-4060-892a-65fedd77f651,45.303329,-122.925797,POINT (-122.92579650878906 45.30332946777344),6531,0,1,residential,6.785987,0.000000,1.0,1.0,0.0
16843,8272,NoSidewalk,NaN,mdbgEoi-XTfZGiKqcWMlqw,1,114.982140,-15.312500,359.096985,-9.832024,fc36440f-38b7-4060-892a-65fedd77f651,45.299858,-122.925591,POINT (-122.92559051513672 45.29985809326172),6518,1,2,residential,29.997796,0.000000,1.0,1.0,0.0
16844,8281,NoSidewalk,NaN,BtyhcErvfI4z31FisBjeQg,1,43.812500,-1.875000,46.910660,-2.319252,fc36440f-38b7-4060-892a-65fedd77f651,45.305443,-122.926605,POINT (-122.92660522460938 45.305442810058594),6516,1,2,residential,42.042123,0.000000,1.0,2.0,0.0
16845,16595,CurbRamp,NaN,--gG93FA53Lkv0H31-9Ruw,1,142.982147,-11.625000,270.257080,-1.495285,fc36440f-38b7-4060-892a-65fedd77f651,45.299377,-122.956367,POINT (-122.95636749267578 45.29937744140625),25,1,3,residential,23.299832,34.000657,0.0,0.0,0.0


In [19]:
gt = pd.read_csv('../../../data/' + city + '/features/' + city + '_ground_truth_labels.csv')

In [20]:
gt

,label_id,ground_truth
0,8,1
1,9,1
2,10,1
3,11,1
4,12,1
...,...,...
2790,17672,0
2791,17674,1
2792,17676,1
2793,17688,1


In [21]:
#merge gt with labels how left
test= pd.merge(df, gt, on='label_id', how='inner')

In [22]:
#rename ground truth to verified
test.rename(columns={'ground_truth': 'verified'}, inplace=True)

In [23]:
test

,label_id,label_type,severity,gsv_panorama_id,zoom,heading,pitch,camera_heading,camera_pitch,user_id,lat,lng,geometry,cluster_id,clustered,cluster_label_count,way_type,distance_to_road,distance_to_intersection,tag,tag_count,description,verified
0,22,CurbRamp,1.0,dslJGqaol13AzhdnPA1epA,1,157.750000,-34.437500,116.283493,-0.733246,a7b29eaa-8032-45f9-be2b-fce49aa0b8c0,45.320690,-122.972359,POINT (-122.97235870361328 45.3206901550293),2235,1,3,residential,17.679072,25.919038,0.0,0.0,0.0,1
1,23,CurbRamp,1.0,kI15oZgTQY8bxvRp-Io7nw,1,152.875000,-34.625000,44.911987,0.174957,a7b29eaa-8032-45f9-be2b-fce49aa0b8c0,45.320698,-122.972198,POINT (-122.97219848632812 45.32069778442383),2281,1,3,residential,14.490867,26.518458,0.0,0.0,0.0,1
2,25,NoCurbRamp,4.0,1IV2IjxbPQem-FE5LwIHFg,1,207.125000,-27.250000,221.050308,1.068039,aa0cf6d5-177b-43e6-8a87-1dc1010892a4,45.297989,-122.979263,POINT (-122.97926330566406 45.29798889160156),5868,1,3,tertiary,14.894478,33.135027,1.0,1.0,0.0,1
3,26,NoCurbRamp,5.0,1IV2IjxbPQem-FE5LwIHFg,1,336.500000,-35.000000,221.050308,1.068039,aa0cf6d5-177b-43e6-8a87-1dc1010892a4,45.298111,-122.979309,POINT (-122.97930908203125 45.29811096191406),6438,1,3,residential,14.826055,30.937320,1.0,1.0,0.0,1
4,29,NoCurbRamp,5.0,q5KyS5ERjAw3HJOHVigtBQ,1,227.937500,-29.562500,263.936096,0.018257,aa0cf6d5-177b-43e6-8a87-1dc1010892a4,45.298023,-122.980125,POINT (-122.9801254272461 45.29802322387695),5984,0,1,residential,19.129094,30.510240,1.0,1.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2790,10608,NoCurbRamp,3.0,Us_OPa9BYRG0w1HDzELG1Q,1,325.500000,0.000000,5.186546,-0.526199,e9273525-f0ad-442b-ab53-ec3fed09cdff,45.301994,-122.976204,POINT (-122.97620391845703 45.30199432373047),4460,1,3,residential,16.887283,28.193084,0.0,0.0,0.0,1
2791,10589,SurfaceProblem,NaN,OELPTaR6ao_tPaSNyd0gxQ,1,24.125000,-33.187500,113.645668,-0.484138,e9273525-f0ad-442b-ab53-ec3fed09cdff,45.304497,-122.978989,POINT (-122.97898864746094 45.30449676513672),14797,0,1,primary,25.736058,0.000000,1.0,3.0,0.0,0
2792,10597,CurbRamp,NaN,pV6ygdJjUolRf7F9prg6hA,1,116.687500,-14.937500,91.113060,-0.234367,e9273525-f0ad-442b-ab53-ec3fed09cdff,45.300274,-122.981087,POINT (-122.98108673095703 45.30027389526367),105,1,4,primary,24.599376,35.915239,0.0,0.0,0.0,1
2793,2837,SurfaceProblem,NaN,oH8cWlFOFrrLk8oiRiQ29w,2,55.812500,-18.250000,271.173340,2.239708,f13fe08b-47c4-49c2-b128-21c6c04948da,45.296898,-122.982178,POINT (-122.982177734375 45.296897888183594),14785,1,2,residential,21.151214,0.000000,1.0,1.0,0.0,1


In [24]:
#if the label_id in gt take it out of df
training = df[~df.label_id.isin(test.label_id)]

In [25]:
training

,label_id,label_type,severity,gsv_panorama_id,zoom,heading,pitch,camera_heading,camera_pitch,user_id,lat,lng,geometry,cluster_id,clustered,cluster_label_count,way_type,distance_to_road,distance_to_intersection,tag,tag_count,description
0,145,NoSidewalk,NaN,0R2JTLibSCNsNb7aGNUzWg,1,319.375000,-19.062500,269.813477,0.678093,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317650,-122.952904,POINT (-122.9529037475586 45.317649841308594),13284,1,2,tertiary,21.228372,0.000000,0.0,0.0,0.0
1,18134,NoSidewalk,4.0,Z6Ja9-4wJxpQZPEnn8KSXA,3,326.437500,-10.906250,323.408020,1.379303,bb78d1ab-85d6-4620-a1de-396a5785080a,45.313820,-122.964005,POINT (-122.96400451660156 45.313819885253906),10678,1,2,tertiary,36.874003,0.000000,1.0,3.0,0.0
2,151,NoSidewalk,NaN,oqg9ULHZul23xfU6DwTRNg,2,285.258942,-11.508928,268.211182,0.926697,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317616,-122.953598,POINT (-122.95359802246094 45.3176155090332),6734,0,1,secondary,34.955005,0.000000,0.0,0.0,0.0
3,153,NoSidewalk,NaN,WWRq8hx_qzBmTiSkSBjEqQ,2,223.651779,-8.741072,268.896179,-0.547432,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317421,-122.954025,POINT (-122.95402526855469 45.317420959472656),13312,0,1,secondary,34.211111,0.000000,0.0,0.0,0.0
4,154,NoSidewalk,NaN,WWRq8hx_qzBmTiSkSBjEqQ,2,279.008942,-9.455358,268.896179,-0.547432,d363619a-53e1-4ef4-a541-bb0278cae7c3,45.317600,-122.954185,POINT (-122.95418548583984 45.31760025024414),13317,0,1,secondary,31.433333,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16842,8222,NoSidewalk,NaN,5KKnsHQgODUBbIRZYiVbqg,2,225.142853,-19.375000,22.380564,-3.511421,fc36440f-38b7-4060-892a-65fedd77f651,45.303329,-122.925797,POINT (-122.92579650878906 45.30332946777344),6531,0,1,residential,6.785987,0.000000,1.0,1.0,0.0
16843,8272,NoSidewalk,NaN,mdbgEoi-XTfZGiKqcWMlqw,1,114.982140,-15.312500,359.096985,-9.832024,fc36440f-38b7-4060-892a-65fedd77f651,45.299858,-122.925591,POINT (-122.92559051513672 45.29985809326172),6518,1,2,residential,29.997796,0.000000,1.0,1.0,0.0
16844,8281,NoSidewalk,NaN,BtyhcErvfI4z31FisBjeQg,1,43.812500,-1.875000,46.910660,-2.319252,fc36440f-38b7-4060-892a-65fedd77f651,45.305443,-122.926605,POINT (-122.92660522460938 45.305442810058594),6516,1,2,residential,42.042123,0.000000,1.0,2.0,0.0
16845,16595,CurbRamp,NaN,--gG93FA53Lkv0H31-9Ruw,1,142.982147,-11.625000,270.257080,-1.495285,fc36440f-38b7-4060-892a-65fedd77f651,45.299377,-122.956367,POINT (-122.95636749267578 45.29937744140625),25,1,3,residential,23.299832,34.000657,0.0,0.0,0.0


In [26]:
training.to_csv('../../../data/' + city + '/processed-labels/training_set_' + city + '.csv', index=False)
test.to_csv('../../../data/' + city + '/processed-labels/test_set_' + city + '.csv', index=False)
df.to_csv('../../../data/' + city + '/processed-labels/labels_full_features_' + city + '.csv', index=False)